# imports

In [ ]:
import os
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import datetime as dt
from scipy import stats
import nltk
import collections
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import math
from pandas import ExcelWriter
import copy
from __future__ import print_function
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF

import pickle

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns

import gensim

In [ ]:
def save_xls(filename, list_dfs, tab_names):
    writer = ExcelWriter(filename, engine='xlsxwriter', options={'strings_to_urls': False})
    for tn, df in zip(tab_names, list_dfs):
#         tn = tn[ : tn.find(',')]
#         tn = tn.replace(':', ' ')
#         print (tn)
        df.to_excel(writer, tn, encoding='utf8', header=True,index=False)
    writer.save()


# Data Addresses

In [ ]:
os.getcwd()

# Read TheRedPill data
1. fasttext preppred
2. len limited
3. repeated threads removed

In [ ]:
preppedDataAddr = ''
TheRedPill = pd.read_csv(preppedDataAddr + 'TheRedPillCleaned.csv', index_col=0)
TheRedPill = TheRedPill.reset_index(drop=True)
print (len(TheRedPill))


# DataDf = TheRedPill.copy()

In [ ]:
dataDf = TheRedPill.copy()

# Stem, stop 

In [ ]:
# manualStopListAddr = '/home/a383khan/Thesis/Feminism-old-doNotDeleteHasFasttext&ControlUsedForLexiconCreation/hateSpeech/mai/data/'
# with open(manualStopListAddr+'manualStopAfterStemList.txt', 'r') as f:
#     manual_stop_words_after_stem = f.readlines()
#     manual_stop_words_after_stem = list(manual_stop_words_after_stem)
#     manual_stop_words_after_stem = [ i.strip() for i in manual_stop_words_after_stem ]

my_stop_before_stem = [
    'comment', 
    'endofcomment', 
    'endoftitlecontent', 
    'gt', 
    'mod',
    'moderator', 
    'post', 
    'subreddit'
]
stop_words_before_stem = stopwords.words('english')
stop_words_before_stem += my_stop_before_stem

stop_words_after_stem = []
ls = nltk.stem.SnowballStemmer(language='english')
for sw in stop_words_before_stem: #stemming even orig stop words so I don't have to run 2 passes for stop word removal
    stop_words_after_stem.append(ls.stem(sw))
stop_words_after_stem = set(stop_words_after_stem)
stop_words_after_stem = list(stop_words_after_stem)

# stop_words_after_stem += manual_stop_words_after_stem

print (len(stop_words_before_stem), len(stop_words_after_stem))

In [ ]:
# import unidecode
import unicodedata

def preprocess(text, remove_stop=False, stem=False, remove_nonNoun=False):
    """
    Since we are interested in topics, we removed stopwords and tokens with
    fewer than two letters, and we optionally only retain nouns which appear in the WordNet
    corpus
    """
    
    tokens = nltk.word_tokenize(text)
    tokens = [token.lower() for token in tokens if len(token) > 1] #same as unigrams

    if remove_nonNoun is True:
        tokens = [ token for token in tokens if token in nouns ]
        
#     if remove_stop is True:
# #         stop_words = stopwords.words('english')
# #         stop_words.append('endoftitlecontent'); stop_words.append('endofcomment'); stop_words.append('gt'); 
#         tokens = [word for word in tokens if word not in stop_words_before_stem]
    
    if stem is True:
        ls = SnowballStemmer(language='english')
        tokens = [ls.stem(word) for word in tokens]
        
    if remove_stop is True: #after stemming removing stop coz I dont know what the original forms were of my custom stop words
#         stop_words = stopwords.words('english')
#         stop_words.append('endoftitlecontent'); stop_words.append('endofcomment'); stop_words.append('gt'); 
        tokens = [word for word in tokens if word not in stop_words_after_stem]
    
        
    ret = ' '.join(tokens)
    ret = ret.replace('`', ' ')

    return ret

In [ ]:
dataDf['text_repreprocessed'] = \
    dataDf['text_preprocessed'].apply(lambda x : preprocess(x, remove_stop=True, stem=True, remove_nonNoun=False))

In [ ]:
print (len(dataDf))

In [ ]:
outAddrRoot = 'data/topicModeling/nmf/standardStopRemoved/TheRedPill/'
if (not os.path.exists(outAddrRoot)):
    os.makedirs(outAddrRoot)
dataDf[['text_repreprocessed']].to_csv(outAddrRoot+'TheRedPillRepreprocessed.csv')

# NMF Code

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

def getTopicTopWords(model, feature_names, n_top_words):
    topicIdToTopWords = {}
    for topic_idx, topic in enumerate(model.components_):
        message = "%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        topicIdToTopWords[topic_idx] = message
#         print(message)
#     print()
    return topicIdToTopWords

def getDocIdToTopicId(nmf, tf, dataDf):
    print ('\nTopic assigned to each document in nmf model ')
    docIdToTopicId = {}
    doc_topic = nmf.transform(tf)
    for n in range(doc_topic.shape[0]): #for each document
        topic_most_pr = doc_topic[n].argmax() #get the name of the topic that has the highest percentage to it 
        docIdToTopicId[dataDf.loc[n]['name']] = topic_most_pr

    return docIdToTopicId


def nmfStuff(df, n_features=1000, n_topics=15, n_top_words=20, 
             ngram_range_lower=1, ngram_range_upper=1):
    print("Loading dataset...")
    t0 = time()
    # data_samples = dataDf['text_preprocessed'].tolist()
    data_samples = df['text_repreprocessed'].tolist()
    print("done in %0.3fs." % (time() - t0))

    # Use tf (raw term count) features for NMF.
    print("Extracting tfidf features for nmf...")

    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, 
                                       max_features=n_features, 
                                      ngram_range = (ngram_range_lower, ngram_range_upper),
                                       stop_words=None)
    tfidf = tfidf_vectorizer.fit_transform(data_samples)

    print()

    print("Fitting nmf models with tfidf features ")
    
    nmf = NMF(n_components=n_topics, 
              random_state=1, alpha=.1, 
              l1_ratio=.5, init='nndsvd')
    nmf.fit(tfidf)
    

    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    # print_top_words(nmf, tf_feature_names, n_top_words)
    topicIdToTopWords = getTopicTopWords(nmf, tfidf_feature_names, n_top_words)
    docIdToTopicId = getDocIdToTopicId(nmf, tfidf, df)

    return topicIdToTopWords, docIdToTopicId, nmf, tfidf, tfidf_vectorizer




# Find best n_topics NMF TheRedPill

In [ ]:
df = dataDf.copy()
k_values = [ 10, 50, 100, 200, 300, 400 ]


# n_topics = 50

In [ ]:
n_topics = 50

In [ ]:
df = dataDf.copy()
df = df.reset_index(drop=True)

In [ ]:
topicIdToTopWords, docIdToTopicId, nmf, tf, tf_vectorizer = nmfStuff(df, n_features=1000, 
    n_topics=n_topics, 
    n_top_words=30, 
 ngram_range_lower=1, ngram_range_upper=1)

In [ ]:
docIdToTopicId = {} #Ordered so that first is best choice, second is second best choice
doc_topic = nmf.transform(tf)
for n in range(doc_topic.shape[0]): #for each document
    temp = list(doc_topic[n])[:]
    
    topic_most_pr = np.argmax(temp) #get the name of the topic that has the highest percentage to it 
    pr = np.max(temp)
    docIdToTopicId[df.loc[n]['name']] = [ topic_most_pr ]
    temp.pop(topic_most_pr)


### How many docs were assigned no, one, and two topics?

In [ ]:
print ('Findings docs assigned one or more topics ')
two = 0
one = 0
for docId in docIdToTopicId:
#     print (docIdToTopicId[docId])
#     print (type(docIdToTopicId[docId]))
    if len(docIdToTopicId[docId]) == 1:
        one += 1
    elif len(docIdToTopicId[docId]) == 2:
        two += 1
    else:
        print ('Error')

print ('Total ', len(df), len(df)/len(dataDf))
print ('Zero ', (len(df) - len(docIdToTopicId)), (len(df) - len(docIdToTopicId))/len(df))
print ('One ', one, one/len(df))
print ('Two ', two, two/len(df))

### Convert docIdToTopicId to topicIdToDocId

In [ ]:
topicIdToDocId = {}
for docId in docIdToTopicId:
    topicIds = docIdToTopicId[docId]
    
    topicId = topicIds[0]
    if topicId in topicIdToDocId:
        topicIdToDocId[topicId].append(docId)
    else:
        topicIdToDocId[topicId] = []
    
    if len(topicIds) == 2:
        topicId = topicIds[1] #comment this section if you only want to consider the topmost choice of topic for a doc to be valid
        if topicId in topicIdToDocId:
            topicIdToDocId[topicId].append(docId)
        else:
            topicIdToDocId[topicId] = []
        
        
print (topicIdToDocId.keys())
print (len(topicIdToDocId))

### Save documents corresponding to each topic as csvs 

In [ ]:
topicIdToDocIdDfList = []
tabNames = []
topicIds = list(topicIdToDocId.keys())
topicIds.sort()
for topicId in topicIds:
    print (topicId)
    tabNames.append(str(topicId))
    topicIdToDocIdDf = df[df['name'].isin(topicIdToDocId[topicId])]
    
    cols = ['name', 'title', 'selftext', 'text_repreprocessed']
    topicIdToDocIdDf = topicIdToDocIdDf[cols]

    #add another row at top with only value in name column = actual topic (limited at top 20 words)
    footnote = pd.DataFrame([[topicIdToTopWords[topicId], '', '', '']], 
                            columns=list(topicIdToDocIdDf.columns))
    topicIdToDocIdDf = pd.concat([footnote, topicIdToDocIdDf])
    topicIdToDocIdDfList.append(topicIdToDocIdDf)
#     break
    
#save this as xlsx file to outAddr
filename = 'unigram'+str(n_topics)+'TopicsTopicIdToDocIdNoThresh.xlsx'
save_xls(outAddr+filename, topicIdToDocIdDfList, tabNames)

In [ ]:
print (len(topicIdToDocIdDfList))

In [ ]:
topicIdToDocIdDf

### What are the topics

In [ ]:
print("\nTopics in NMF model:")
x = list(topicIdToTopWords.keys())
x.sort()
for i in x:
    print ("Topic #", i, ": ", topicIdToTopWords[i])
    print ()



### Get document id to topic id dictionary of probabilties

In [ ]:
doc_topic = nmf.transform(tf)

In [ ]:
docIdToTopicIdProb = {}
for n in range(doc_topic.shape[0]): #for each document
    docName = dataDf.loc[n]['name']
    print (docName)
    docIdToTopicIdProb[docName] = {}
    topicIds = range(0, len(doc_topic[n]))
    for topicId in topicIds:
#         print (topicId)
        docIdToTopicIdProb[docName][topicId] = doc_topic[n][topicId]
#     break

### Plot the distribution, y axis number of documents, x importance to a document. will have as many distributions as n_topics - andys distribution


In [ ]:
if (not os.path.exists(outAddr+'AndysPlotsImpToADocVsNoDocsForEachTopic/')):
    os.makedirs(outAddr+'AndysPlotsImpToADocVsNoDocsForEachTopic/')


In [ ]:
import matplotlib.pyplot as plt

topicIds = sorted(list(topicIdToDocId.keys()))
xs = []
group_labels = []
for topicId in topicIds:
    print (topicId)
#     if topicId < 1:
#         continue
    group_label = 'Topic '+ str(topicId) + ': ' + topicIdToTopWords[topicId]
    x = []
    for docId in docIdToTopicIdProb:
        x.append(docIdToTopicIdProb[docId][topicId])
    
    xs.append(x)
    group_labels.append(group_label)
    
    x = list(filter(lambda a: a > 0, x))
    x = list(filter(lambda a: a >= 0.01, x)) #filter out importances of less than 0.02 for clarity 

    fig = sns.distplot(x, hist=True, kde=True, rug=False, bins=10)
    fig.set_title(group_label)
    fig = fig.get_figure()
    fig.savefig(outAddr+'AndysPlotsImpToADocVsNoDocsForEachTopic/'+'topic'+str(topicId)+'.png')
    
    plt.clf()